In [18]:
from pathlib import Path

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

import geojson
import shapely
import folium

## Data from Aguirre-Gutiérrez et al. (2021) study in 6 tropical countries.

Extract boundaries of the different plots from the pixel coordinates. Only X and Y coordinates are provided so we need to first find the UTM zone for each country, then project into the WGS84 CRS.

***Missing data for NXV plots (less than 70% basal area) and MNG-04.*** NXV-01 and NXV-10 are forest savannah transistional zones so that might explain it.

### Method
- Resampled 20m Sentinel-2 bands to 10m spatial resolution using bilinear interpolation. 
- Sentinel-2 60 m resolution bands (B01, B09, B10) ignored as they were designed for cirrus, water vapour and cloud detection.
- Band 8A not used as it covers an overlapping spectral window with band 8 and has a lower spatial resolution. 
- Vegetation indices may increase prediction accuracy when modelling community weighted traits (Wallis et al., 2019), we calculated three of them (Table 3) which we hypothesised to inform trait distributions given their association with chlorophyll and nutrient levels in the leaves and their use of the visible-to-red edge spectral bands.

In [42]:
# Provide plot centroid locations from Aguirre-Gutiérrez (2021) to find UTM zone for X,Y coordinates.
PLOT_LOCATIONS = {
    'AEP02': {'centroid': (145.586,	-17.146)},
    'AEP03': {'centroid': (145.592, -17.088)},
    'DRO01': {'centroid': (145.430, -16.103)},
    'ROB06': {'centroid': (145.630, -17.121)},
    'ANK01': {'centroid': (-2.696, 5.268)},
    'ANK03': {'centroid': (-2.692, 5.271)},
    'BOB1': {'centroid': (-1.339, 6.691)},
    'BOB2': {'centroid': (-1.319, 6.704)},
    'LPG-01': {'centroid': (11.574, -0.174)},
    'LPG-02': {'centroid': (11.615, -0.216)},
    # 'MNG-04': {'centroid': (9.324, 0.577)},
    'Mondah': {'centroid': (9.324, 0.577)},
    # 'NXV01': {'centroid': (-52.352, -14.708)},
    # 'NXV02': {'centroid': (-52.351, -14.701)},
    'VCR02': {'centroid': (-52.168, -14.832)},
    # 'NXV10': {'centroid': (-52.353, -14.713)},
    '261_10': {'centroid': (-55.005, -3.019)},
    '261_9': {'centroid': (-55.015, -3.040)},
    '363_6': {'centroid': (-54.956, -3.337)},
    '363_3': {'centroid': (-54.963, -3.297)},
    '363_7': {'centroid': (-54.961, -3.321)},
    'ESP01': {'centroid': (-71.595, -13.176)},
    'PAN02': {'centroid': (-71.263, -12.650)},
    'SPD01': {'centroid': (-71.542, -13.047)},
    'SPD02': {'centroid': (-71.537, -13.049)},
    'TRU04': {'centroid': (-71.589, -13.106)},
    'WAY01': {'centroid': (-71.587, -13.191)},
    'ACJ01': {'centroid': (-71.632, -13.147)},
    'PAN03': {'centroid': (-71.274, -12.638)},
    'TAM05': {'centroid': (-69.271, -12.830)},
    'TAM06': {'centroid': (-69.296, -12.839)},
    'SAF-01': {'centroid': (117.619, 4.732)}, # Reversed from paper.
    'SAF-02': {'centroid': (117.617, 4.739)}, # Reversed from paper.
    'SAF-03': {'centroid': (117.588, 4.691)}, # Reversed from paper.
    'SAF-04': {'centroid': (117.700, 4.765)}, # Reversed from paper.
    'DAN-04': {'centroid': (117.796, 4.951)}, # Reversed from paper.
    'DAN-05': {'centroid': (117.793, 4.953)}, # Reversed from paper.
    'MLA-01': {'centroid': (116.970, 4.747)}, # Reversed from paper.
    'MLA-02': {'centroid': (116.950, 4.754)}, # Reversed from paper.
}

In [43]:
len(PLOT_LOCATIONS)

35

In [44]:
def find_utm_zone(longitude: float, latitude: float) -> dict:
    """
    Identify UTM zone and EPSG code based on longitude and latitude.
    
    Parameters:
    -----------
    longitude : float
        Longitude in decimal degrees.
    latitude : float
        Latitude in decimal degrees.
    
    Returns:
    --------
    dict: Detailed UTM zone information.
    """
    # Calculate UTM zone
    utm_zone = int((longitude + 180) / 6) + 1
    
    # Determine hemisphere
    hemisphere = 'N' if latitude >= 0 else 'S'
    
    # EPSG code generation
    epsg_base = 32600 if hemisphere == 'N' else 32700
    epsg_code = epsg_base + utm_zone
    
    return {
        'utm_zone': f"{utm_zone}{hemisphere}",
        'epsg_code': epsg_code,
        'full_proj4_string': f"+proj=utm +zone={utm_zone} +{hemisphere.lower()}hem"
    }

def convert_coordinates(df, input_epsg, x_col='X', y_col='Y'):
    """
    Convert projected coordinates to geographic coordinates
    
    Parameters:
    -----------
    gdf : geopandas.GeoDataFrame
        DataFrame containing X and Y coordinates
    x_col : str, optional
        Name of the X coordinate column (default 'X')
    y_col : str, optional
        Name of the Y coordinate column (default 'Y')
    input_epsg : int, optional
        EPSG code of the input coordinate system (default 32610 - UTM Zone 10N)
    
    Returns:
    --------
    geopandas.GeoDataFrame
        GeoDataFrame with converted coordinates
    """
    # Create GeoDataFrame with original coordinates
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[x_col], df[y_col]),
        crs=f'EPSG:{input_epsg}'
    )
    
    # Convert to WGS84 (latitude/longitude)
    gdf_wgs84 = gdf.to_crs('EPSG:4326')
    
    return gdf_wgs84

In [45]:
def extract_plot_geometry(gdf: gpd.GeoDataFrame, plot_id: str, boundary: bool = True) -> geojson.Feature:
    """Extract the polygon for a given plot ID from the GeoDataFrame."""
    # Filter the valid_plots DataFrame for the current plot_ID
    plot_mask = gdf['New_Plot'].str.contains(plot_id, case=False, na=False)

    # Extract the variable with the most data points for the plot to get the full size.
    # Will need to make sure missing data is correctly handled by the ML model.
    var = gdf[plot_mask].groupby('variable').size().sort_values(ascending=False).index[0]
    var_mask = gdf['variable'] == var
    
    points = list(gdf[plot_mask & var_mask]['geometry'])
    # Create Polygon geometry from plot points – convex hull encloses all points.
    if boundary:
        # Create a convex hull from the points
        geometry = shapely.geometry.MultiPoint(points).convex_hull
    else:
        # Create a MultiPoint geometry from the points
        geometry = shapely.geometry.MultiPoint(points)
    
    # Convert the polygon to GeoJSON format (returns a string)
    geometry_geojson = shapely.to_geojson(geometry)
    return geojson.loads(geometry_geojson) # Convert to a dictionary

In [46]:
dpath = Path('/Users/campbelli/Documents/geofm-plant-traits/data')
df = pd.read_csv(dpath / 'Master_Table_Modelling_LOWCORRELATIONS_wB8A.csv')
# Select only the pixels with 70% of the basal area covered and only community-weighted means.
cwms = df[(df['PercentCovered'] == '70') & (df['Type'] == 'CWMean')]
cwms.loc[:, 'TraitValue'] = pd.to_numeric(cwms['TraitValue'], errors='raise')

/var/folders/g5/h0yjdszs0pz_qr3k5z8bjjq80000gp/T/ipykernel_4322/759108687.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dpath / 'Master_Table_Modelling_LOWCORRELATIONS_wB8A.csv')


In [47]:
# Create a new colum for the EPSG code.
cwms.loc[:, 'EPSG_code'] = np.nan

# Each location has a different UTM zone. Find the UTM zone using the plot 
# centroid and add to cwms.
for name, loc in PLOT_LOCATIONS.items():
    lon, lat = loc['centroid']
    zone_info = find_utm_zone(lon, lat)
    plot_mask = cwms['New_Plot'].str.contains(name, case=False, na=False)
    cwms.loc[plot_mask, 'EPSG_code'] = zone_info['epsg_code']

# Group by EPSG code and convert coordinates to WGS84.
# Assumes each pixel from a plot is contained within a single UTM zone.
gdf_wgs84 = cwms.groupby('EPSG_code').apply(
    lambda x: convert_coordinates(x, input_epsg=x.name, x_col='X', y_col='Y')
)

/var/folders/g5/h0yjdszs0pz_qr3k5z8bjjq80000gp/T/ipykernel_4322/2201525095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cwms.loc[:, 'EPSG_code'] = np.nan
/var/folders/g5/h0yjdszs0pz_qr3k5z8bjjq80000gp/T/ipykernel_4322/2201525095.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gdf_wgs84 = cwms.groupby('EPSG_code').apply(


In [180]:
pixel_coords = gdf_wgs84.reset_index(drop=True)[['New_Plot','geometry']]

In [181]:
pixel_coords['Lat'], pixel_coords['Lon'] = pixel_coords['geometry'].y, pixel_coords['geometry'].x
pixel_coords.drop(columns=['geometry'], inplace=True)
pixel_coords.set_index('New_Plot', inplace=True)

# Drop repeat indices.
pixel_coords.drop_duplicates(inplace=True)

In [190]:
metadata_path = Path('/Users/campbelli/Documents/geofm-plant-traits/data/metadata')
pixel_coords.to_csv(metadata_path / 'pixel_coords.csv', index=True)

In [183]:
# What's up with Mondah? – is that MNG-04, Gabon?
print([label for label in cwms['New_Plot'].unique() if 'Mondah' in label])

# And NXV does not have any pixel values for plots with at least 70% basal area covered...
df[df['New_Plot'].str.contains('NXV10', case=False, na=False)]['PercentCovered'].unique()

['Mondah100', 'Mondah101', 'Mondah102', 'Mondah105', 'Mondah106', 'Mondah107', 'Mondah108', 'Mondah109', 'Mondah110', 'Mondah111', 'Mondah112', 'Mondah113', 'Mondah114', 'Mondah115', 'Mondah118', 'Mondah119', 'Mondah120', 'Mondah121', 'Mondah122', 'Mondah123', 'Mondah124', 'Mondah125', 'Mondah128', 'Mondah131', 'Mondah134', 'Mondah135', 'Mondah136', 'Mondah141', 'Mondah144', 'Mondah145', 'Mondah146', 'Mondah147', 'Mondah148', 'Mondah149', 'Mondah15', 'Mondah150', 'Mondah151', 'Mondah152', 'Mondah153', 'Mondah154', 'Mondah157', 'Mondah158', 'Mondah159', 'Mondah16', 'Mondah160', 'Mondah165', 'Mondah166', 'Mondah167', 'Mondah17', 'Mondah18', 'Mondah19', 'Mondah21', 'Mondah22', 'Mondah23', 'Mondah24', 'Mondah28', 'Mondah29', 'Mondah30', 'Mondah31', 'Mondah35', 'Mondah36', 'Mondah37', 'Mondah40', 'Mondah41', 'Mondah42', 'Mondah43', 'Mondah44', 'Mondah45', 'Mondah46', 'Mondah47', 'Mondah48', 'Mondah49', 'Mondah53', 'Mondah54', 'Mondah55', 'Mondah57', 'Mondah58', 'Mondah59', 'Mondah60', 'Mond

array(['ANY', '50'], dtype=object)

In [184]:
gdf_wgs84.reset_index(drop=True, inplace=True)

In [56]:
plots_dpath = Path('/Users/campbelli/Documents/mini-project/data/boundaries')
with open(plots_dpath / 'tropical_plot_JAG_2021.geojson') as f:
    tropical_plots = geojson.load(f)

In [67]:
plots_list = []
for feature in tropical_plots['features']:
    # convert GeoJSON to shapely polygon
    plots_list.append(shapely.from_geojson(str(feature)))

In [ ]:
# Save list of polygons to a shape file.
plots_gdf = gpd.GeoDataFrame(
    geometry=plots_list,
    crs='EPSG:4326',
    index=[feature['properties']['Plot_ID'] for feature in tropical_plots['features']]
)
plots_gdf.drop('363_7', inplace=True) # Drop LineString object – points all in a line.
plots_gdf.to_file(plots_dpath / 'tropical_plots_JAG_2021.shp', driver='ESRI Shapefile')

In [59]:
# Create a GeoJSON FeatureCollection of plot geometries.
plot_shapes = []
boundary = False # Boundary Polygon else saves as MultiPoint geometry.

for plot_id in PLOT_LOCATIONS.keys():
  # Create GeoJSON geometry for the boundary of each plot containing all points.
  boundary_geometry = extract_plot_geometry(gdf_wgs84, plot_id, boundary=boundary)

  # Create a Feature for each plot
  feature = {
      "type": "Feature",
      "properties": {
          "Plot_ID": plot_id,
      },
      "geometry": boundary_geometry # Convert to GeoJSON and loads from string.
  }
  plot_shapes.append(feature)

# Create the GeoJSON object
geojson_plots = {
  "type": "FeatureCollection",
  "features": plot_shapes
}

In [53]:
# Search feature collection for plot with Plot_ID 'Mondah'
for feature in geojson_plots['features']:
    if feature['properties']['Plot_ID'] == 'Mondah':
        feature['properties']['Plot_ID'] = 'MNG-04'
        break

In [ ]:
# Save the GeoJSON data to a file
opath = dpath / 'boundaries'
ofname = 'tropical_plot_JAG_2021.geojson' if boundary else 'tropical_plot_pixels_JAG_2021.geojson'
if not (plots_dpath / ofname).exists():
  with open(plots_dpath / ofname, 'w') as f:
    geojson.dump(geojson_plots, f)

In [65]:
geojson_plots['features'][2]['geometry']['coordinates'][0]

[145.430681, -16.10353]

In [66]:
# Create map centred on first plot.
coords = geojson_plots['features'][2]['geometry']['coordinates'][0]#[0]
m = folium.Map(location=[coords[1], coords[0]], zoom_start=16)

for feature in geojson_plots['features']:
    plot_id = feature['properties']['Plot_ID']
    folium.GeoJson(
        feature
    ).add_child(folium.Popup(f"Plot ID: {plot_id}")).add_to(m)
# folium.LayerControl().add_to(m)
m